In [ ]:
import os
import shutil
from PIL import Image # Para verificar dimensões da imagem para os bboxes
import matplotlib.pyplot as plt # Para visualização de bboxes
import matplotlib.patches as patches # Para desenhar retângulos
import cv2 # Usado na seção de verificação opcional, pode ser necessário para ler imagens

In [ ]:
import os
import shutil

# --- Configurações de Caminho ---
FINAL_MERGED_IMAGE_DIR = "all_dataset_images" # Nome da pasta para juntar todas as imagens

# Certifica-se de que a pasta de destino final existe
if not os.path.exists(FINAL_MERGED_IMAGE_DIR):
    os.makedirs(FINAL_MERGED_IMAGE_DIR)
    print(f"Pasta de destino '{FINAL_MERGED_IMAGE_DIR}' criada.")

# Diretórios de onde as imagens serão COLETADAS
image_dirs = [
    'PART_1/PART_1/images',
    'PART_2/PART_2/images',
    'PART_3/PART_3/images'
]

# Extensões de arquivo de imagem que serão consideradas
IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png')

# --- Processo de Junção de Imagens ---

print("\nIniciando a cópia das imagens...")
copied_image_count = 0

for source_dir in image_dirs:
    # Verifica se o diretório de origem existe antes de tentar listar
    if not os.path.exists(source_dir):
        print(f"Aviso: Diretório de origem '{source_dir}' não encontrado. Pulando...")
        continue

    print(f"Processando diretório: {source_dir}")
    for fname in os.listdir(source_dir):
        # Verifica se o arquivo é uma imagem pela extensão
        if fname.lower().endswith(IMAGE_EXTENSIONS):
            source_image_path = os.path.join(source_dir, fname)
            base_name, extension = os.path.splitext(fname)
            destination_image_name = f"{base_name}{extension}"
            destination_image_path = os.path.join(FINAL_MERGED_IMAGE_DIR, destination_image_name)

            try:
                shutil.copy2(source_image_path, destination_image_path)
                # print(f"Copiado: {source_image_path} -> {destination_image_path}")
                copied_image_count += 1
            except Exception as e:
                print(f"ERRO ao copiar '{source_image_path}': {e}")

print(f"\nProcesso concluído. Total de {copied_image_count} imagens copiadas para '{FINAL_MERGED_IMAGE_DIR}'.")

In [ ]:
import os
import shutil

# --- Configurações de Caminho ---
FINAL_MERGED_ANNOTATION = "all_dataset_annotations" # Nome da pasta para juntar todas as anotações

# Certifica-se de que a pasta de destino final existe
if not os.path.exists(FINAL_MERGED_ANNOTATION):
    os.makedirs(FINAL_MERGED_ANNOTATION)
    print(f"Pasta de destino '{FINAL_MERGED_ANNOTATION}' criada.")

# Diretórios de onde as anotações serão COLETADAS
ANNOTATIONs = ['PART_1/PART_1/6categories']

# Extensões de arquivo de imagem que serão consideradas
ANNOTATIONs_EXTENSION = ('.txt')

# --- Processo de Junção de anotações ---

print("\nIniciando a cópia das anotações...")
copied_image_count = 0

for source_dir in ANNOTATIONs:
    # Verifica se o diretório de origem existe antes de tentar listar
    if not os.path.exists(source_dir):
        print(f"Aviso: Diretório de origem '{source_dir}' não encontrado. Pulando...")
        continue

    print(f"Processando diretório: {source_dir}")
    for fname in os.listdir(source_dir):
        # Verifica se o arquivo é uma imagem pela extensão
        if fname.lower().endswith(ANNOTATIONs_EXTENSION):
            source_image_path = os.path.join(source_dir, fname)
            base_name, extension = os.path.splitext(fname)
            destination_image_name = f"{base_name}{extension}"
            destination_image_path = os.path.join(FINAL_MERGED_ANNOTATION, destination_image_name)

            try:
                shutil.copy2(source_image_path, destination_image_path)
                # print(f"Copiado: {source_image_path} -> {destination_image_path}")
                copied_image_count += 1
            except Exception as e:
                print(f"ERRO ao copiar '{source_image_path}': {e}")

print(f"\nProcesso concluído. Total de {copied_image_count} anotações copiadas para '{FINAL_MERGED_ANNOTATION}'.")

In [ ]:
import os

# Define o caminho para a pasta de anotações e o arquivo de nomes das classes
annotations_dir = "all_dataset_annotations"
names_file = "PART_1/PART_1/6categories.names"
# Sugiro um novo nome para o diretório de saída para não sobrescrever o anterior
output_dir = "cleaned_dataset_annotations_no_bouy" 

# Cria o diretório de saída se não existir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 1. Ler o arquivo de nomes das classes para mapear nomes para índices
class_names = []
with open(names_file, 'r') as f:
    class_names = [line.strip() for line in f.readlines()]

# 2. Identificar o índice da classe a ser REMOVIDA ("bouy")
bouy_idx = -1
try:
    bouy_idx = class_names.index("bouy")
except ValueError:
    print(f"Erro: A classe 'bouy' não foi encontrada em {names_file}.")
    print(f"Classes lidas: {class_names}")
    exit()

print(f"Índice da classe 'bouy' a ser removida: {bouy_idx}")
print(f"Todas as outras classes serão mantidas: {', '.join([name for i, name in enumerate(class_names) if i != bouy_idx])}")

# 3. Iterar sobre cada arquivo de anotação no diretório
for filename in os.listdir(annotations_dir):
    if filename.endswith(".txt"):
        input_filepath = os.path.join(annotations_dir, filename)
        output_filepath = os.path.join(output_dir, filename)
        
        kept_annotations = [] # Renomeado para maior clareza
        
        with open(input_filepath, 'r') as infile:
            for line in infile:
                parts = line.strip().split()
                if not parts:
                    continue
                
                try:
                    class_id = int(parts[0])
                    # Manter a linha APENAS SE a classe NÃO FOR 'bouy'
                    if class_id != bouy_idx:
                        kept_annotations.append(line)
                except ValueError:
                    print(f"Aviso: Linha inválida ignorada em {filename}: {line.strip()}")
                    continue
        
        # 4. Escrever as anotações filtradas em um novo arquivo
        if kept_annotations:
            with open(output_filepath, 'w') as outfile:
                for annotation in kept_annotations:
                    outfile.write(annotation)
            print(f"Arquivo filtrado salvo em: {output_filepath}")
        else:
            # Isso pode acontecer se um arquivo SÓ tinha anotações de 'bouy'
            print(f"Nenhuma anotação mantida para o arquivo: {filename} (após remover 'bouy'). Nenhum arquivo de saída gerado.")

print("\nLimpeza (remoção de 'bouy') concluída!")

In [ ]:
import os

# --- Configurações ---
# Diretório com TODAS as imagens (com base no seu notebook, parece ser este)
images_dir = "all_dataset_images" 

# Diretório com as anotações JÁ LIMPAS (o resultado do seu script anterior)
annotations_dir = "cleaned_dataset_annotations_no_bouy"

# --- Lógica do Script ---

# 1. Obter a lista de nomes base das anotações existentes
# Usamos um 'set' para uma busca muito mais rápida
annotation_basenames = set()
for ann_file in os.listdir(annotations_dir):
    if ann_file.endswith(".txt"):
        # Pega o nome do arquivo sem a extensão .txt (ex: 'img_001')
        basename = os.path.splitext(ann_file)[0]
        annotation_basenames.add(basename)

print(f"Encontradas {len(annotation_basenames)} anotações limpas em '{annotations_dir}'.")

# 2. Identificar imagens que não têm uma anotação correspondente
images_to_delete = []
for img_file in os.listdir(images_dir):
    # Pega o nome do arquivo de imagem sem a extensão (ex: 'img_001')
    img_basename = os.path.splitext(img_file)[0]
    
    # Se o nome base da imagem NÃO está na lista de nomes de anotações, ela deve ser apagada
    if img_basename not in annotation_basenames:
        images_to_delete.append(os.path.join(images_dir, img_file))

# 3. Revisão e Confirmação (PASSO DE SEGURANÇA)
if not images_to_delete:
    print("\nNenhuma imagem para apagar. Todas as imagens têm anotações correspondentes.")
else:
    print(f"\nAs seguintes {len(images_to_delete)} imagens serão APAGADAS pois não possuem anotações:")
    print(images_to_delete)
    # Mostra as primeiras 10 imagens para o usuário ter uma ideia
    for i, image_path in enumerate(images_to_delete[:10]):
        print(f" - {os.path.basename(image_path)}")
    if len(images_to_delete) > 10:
        print(f"   ... e mais {len(images_to_delete) - 10} outras.")

    # Pede confirmação ao usuário antes de apagar qualquer coisa
    # Mude para 's' se seu sistema for em português
    user_input = input("\nVocê tem certeza que deseja apagar esses arquivos? (s/n): ").lower()
    
    # 4. Apagar os arquivos se confirmado
    if user_input == 's' or user_input == 'y':
        deleted_count = 0
        for image_path in images_to_delete:
            try:
                os.remove(image_path)
                deleted_count += 1
            except OSError as e:
                print(f"Erro ao apagar o arquivo {image_path}: {e}")
        
        print(f"\nProcesso concluído. {deleted_count} imagens foram apagadas.")
    else:
        print("\nOperação cancelada pelo usuário. Nenhum arquivo foi apagado.")